In [2]:
import pandas as pd
import requests

In [6]:
missions_df = pd.read_csv('../datasets/mission_launches.csv')
locations_df = pd.read_csv('../datasets/coordinates.csv')

In [4]:
missions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    4324 non-null   int64 
 1   Unnamed: 0      4324 non-null   int64 
 2   Organisation    4324 non-null   object
 3   Location        4324 non-null   object
 4   Date            4324 non-null   object
 5   Detail          4324 non-null   object
 6   Rocket_Status   4324 non-null   object
 7   Price           964 non-null    object
 8   Mission_Status  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


In [30]:
locations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   10 non-null     object 
 1   Longitude  10 non-null     float64
 2   Latitude   10 non-null     float64
dtypes: float64(2), object(1)
memory usage: 372.0+ bytes


In [7]:
df = pd.merge(missions_df, locations_df, on='Location', how='left')


In [55]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Latitude,Longitude
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,30.919409,-87.314059
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,35.000000,105.000000
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,30.056755,-94.098334
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,46.000000,63.250000
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,28.394020,-80.603151


In [8]:
print(df[['Date','Latitude', 'Longitude']])

                            Date   Latitude   Longitude
0     Fri Aug 07, 2020 05:12 UTC  30.919409  -87.314059
1     Thu Aug 06, 2020 04:01 UTC  35.000000  105.000000
2     Tue Aug 04, 2020 23:57 UTC  30.056755  -94.098334
3     Thu Jul 30, 2020 21:25 UTC  46.000000   63.250000
4     Thu Jul 30, 2020 11:50 UTC  28.394020  -80.603151
...                          ...        ...         ...
4319  Wed Feb 05, 1958 07:33 UTC  28.394020  -80.603151
4320  Sat Feb 01, 1958 03:48 UTC  28.394020  -80.603151
4321  Fri Dec 06, 1957 16:44 UTC  28.394020  -80.603151
4322  Sun Nov 03, 1957 02:30 UTC  46.000000   63.250000
4323  Fri Oct 04, 1957 19:28 UTC  46.000000   63.250000

[4324 rows x 3 columns]


In [12]:
import requests
import pandas as pd
from datetime import datetime
import time

# API configuration
api_key = 'FBZN4WMJ6NEP97NGXKV2NTL3N'  # Replace with your Visual Crossing API key
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

# Initialize a counter and a daily limit
request_counter = 0
daily_limit = 1000

def fetch_weather(lat, lon, date, time):
    """Fetch weather data for a specific hour from Visual Crossing API."""
    global request_counter
    if request_counter >= daily_limit:
        print("Daily limit reached. Waiting for the next day...")
        return None
    
    # Combine date and time for the API request
    datetime_str = f"{date}T{time}"
    full_url = f"{url}{lat},{lon}/{datetime_str}?key={api_key}&unitGroup=metric"
    
    print(f"Request URL: {full_url}")
    response = requests.get(full_url)
    request_counter += 1
    print(f"Request #{request_counter} - Status Code: {response.status_code}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error Response: {response.text}")
        return None

# Assuming your 'Date' column includes both date and time information like '2024-09-01 14:00'
# Convert 'Date' to datetime and extract date and time separately
df['Date'] = pd.to_datetime(df['Date'], format='%a %b %d, %Y %H:%M %Z', errors='coerce').dt.strftime('%Y-%m-%d')
df['Date'] = pd.to_datetime(df['Date'], format='%a %b %d, %Y %H:%M %Z', errors='coerce')
df['Time'] = df['Date'].dt.strftime('%H:%M')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Initialize a list to store weather data
weather_data = []

# Iterate over rows and fetch weather data for the specific hour
for index, row in df.iterrows():
    print(f"Processing row {index + 1}...")
    date_str = row['Date']
    time_str = row['Time']
    weather = fetch_weather(row['Latitude'], row['Longitude'], date_str, time_str)
    
    if weather is None:
        print(f"Skipping row {index + 1} due to error or rate limit.")
        weather_info = {
            'Temperature': None,
            'Humidity': None,
            'Wind Speed': None,
            'Precipitation': None,
            'Weather Description': None
        }
    else:
        # Extract relevant weather information
        try:
            current_weather = weather['currentConditions']
            weather_info = {
                'Temperature': current_weather.get('temp', None),
                'Humidity': current_weather.get('humidity', None),
                'Wind Speed': current_weather.get('windspeed', None),
                'Precipitation': current_weather.get('precip', None),
                'Weather Description': current_weather.get('conditions', None)
            }
            print(f"Weather data for row {index + 1}: {weather_info}")
        except KeyError:
            print(f"Error extracting data for row {index + 1}. Response: {weather}")
            weather_info = {
                'Temperature': None,
                'Humidity': None,
                'Wind Speed': None,
                'Precipitation': None,
                'Weather Description': None
            }
    
    weather_data.append(weather_info)

    # Rate limit handling (to be more polite to the API, you can add delays)
    if request_counter % 100 == 0:
        print(f"Reached {request_counter} requests. Sleeping for a short while...")
        time.sleep(60)  # Sleep for 1 minute to avoid hitting the rate limit too quickly

# Convert weather data to a DataFrame
weather_df = pd.DataFrame(weather_data)

# Concatenate with the original DataFrame
result_df = pd.concat([df, weather_df], axis=1)

# Save to a new CSV file
result_df.to_csv('dataset_with_weather.csv', index=False)

print("Data retrieval complete. Saved to 'dataset_with_weather.csv'.")


Processing row 1...
Request URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/30.919409,-87.314059/nanTnan?key=FBZN4WMJ6NEP97NGXKV2NTL3N&unitGroup=metric
Request #1 - Status Code: 400
Error Response: Bad API Request:Start date time value nanTnan cannot be parsed. Format must be yyyy-M-d['T'H:m:s][.SSS][X]
Skipping row 1 due to error or rate limit.
Processing row 2...
Request URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/35.0,105.0/nanTnan?key=FBZN4WMJ6NEP97NGXKV2NTL3N&unitGroup=metric


KeyboardInterrupt: 

In [63]:
import requests

API_KEY = '6302bcf676aba098e3d8cd4a6ffe9e0f'  # Replace with your actual OpenWeather API key

# Test API call
response = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q=London&appid={API_KEY}')

if response.status_code == 200:
    print("API Key is working.")
else:
    print(f"API Key is not working. Status code: {response.status_code}")
    print(response.json())


API Key is not working. Status code: 401
{'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}


In [38]:
# Define a function to get weather data for a given latitude and longitude
def get_weather_data(lat, lon):
    try:
        url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true'
        response = requests.get(url)
        data = response.json()
        
        if 'current_weather' not in data:
            print(f"No weather data found for coordinates: {lat}, {lon}")
            return None, None, None, None
        
        current_weather = data.get('current_weather', {})
        
        temperature = current_weather.get('temperature', None)
        weather_code = current_weather.get('weathercode', None)
        precipitation = current_weather.get('precipitation', None)
        wind_speed = current_weather.get('windspeed_10m', None)
        
        return temperature, weather_code, precipitation, wind_speed
    except Exception as e:
        print(f"Error fetching weather data for coordinates {lat}, {lon}: {e}")
        return None, None, None, None

# Create a new DataFrame for weather data
weather_data = []

# Loop through each row in the original DataFrame
for index, row in df.iterrows():
    print(f"Processing row {index} with coordinates ({row['Latitude']}, {row['Longitude']})")
    temp, code, precip, wind = get_weather_data(row['Latitude'], row['Longitude'])
    weather_data.append([temp, code, precip, wind])
    print(f"Weather data for row {index}: Temperature={temp}, Weather_Code={code}, Precipitation={precip}, Wind_Speed={wind}")

# Convert weather data to a DataFrame
weather_df = pd.DataFrame(weather_data, columns=['Temperature', 'Weather_Code', 'Precipitation', 'Wind_Speed'])

# Concatenate the original DataFrame with the new weather DataFrame
updated_df = pd.concat([df, weather_df], axis=1)

# Save the updated DataFrame to a new CSV file
updated_df.to_csv('updated_file_with_weather.csv', index=False)

print("Weather data integration complete. Saved to 'updated_file_with_weather.csv'.")


Processing row 0 with coordinates (28.574517350132584, -80.65199418806188)
Weather data for row 0: Temperature=30.1, Weather_Code=1, Precipitation=None, Wind_Speed=None
Processing row 1 with coordinates (40.98685246952713, 100.2152295878324)
Weather data for row 1: Temperature=28.5, Weather_Code=1, Precipitation=None, Wind_Speed=None
Processing row 2 with coordinates (25.990266, -97.182972)
Weather data for row 2: Temperature=30.5, Weather_Code=95, Precipitation=None, Wind_Speed=None
Processing row 3 with coordinates (45.963951, 63.305039)
Weather data for row 3: Temperature=22.0, Weather_Code=0, Precipitation=None, Wind_Speed=None
Processing row 4 with coordinates (28.492075, -80.578018)
Weather data for row 4: Temperature=31.2, Weather_Code=1, Precipitation=None, Wind_Speed=None
Processing row 5 with coordinates (38.848494, 111.60794)
Weather data for row 5: Temperature=15.3, Weather_Code=1, Precipitation=None, Wind_Speed=None
Processing row 6 with coordinates (45.964473, 63.305264)


KeyboardInterrupt: 

In [ ]:

# Load your DataFrame
df = pd.read_csv('locations.csv')

# Replace 'YOUR_API_KEY' with your actual OpenCage API key
API_KEY = '6363f1bfef24435297c4e335a10a9d64'

def get_coordinates(location):
    try:
        url = f'https://api.opencagedata.com/geocode/v1/json?q={location}&key={API_KEY}'
        response = requests.get(url)
        data = response.json()
        if data['results']:
            geometry = data['results'][0]['geometry']
            return geometry['lat'], geometry['lng']
        else:
            print(f"No coordinates found for location: {location}")
            return None, None
    except Exception as e:
        print(f"Error fetching coordinates for location {location}: {e}")
        return None, None

# Apply the function to each location and add longitude and latitude columns
df[['Latitude', 'Longitude']] = df['Location'].apply(lambda loc: pd.Series(get_coordinates(loc)))

# Save the updated DataFrame
df.to_csv('locations_with_coordinates.csv', index=False)

print("Coordinates added successfully. Saved to 'locations_with_coordinates.csv'.")
